In [ ]:
!pip install timm
!pip install coremltools

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 549.1/549.1 KB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 8.5 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 13.2 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import torch
import coremltools as ct

#model_type = "DPT_Large"     # MiDaS v3 - Large     (highest accuracy, slowest inference speed)
#model_type = "DPT_Hybrid"   # MiDaS v3 - Hybrid    (medium accuracy, medium inference speed)
model_type = "MiDaS_small"  # MiDaS v2.1 - Small   (lowest accuracy, highest inference speed)

class Model(torch.nn.Module):

  def __init__(self, model_type):
    super(Model, self).__init__()

    self.mean = torch.tensor([0.485, 0.456, 0.406])
    self.std = torch.tensor([0.229, 0.224, 0.225])
    self.midas = torch.hub.load('intel-isl/MiDaS', model_type)

  def forward(self, x):
    x = x / 255.0
    x = (x - 0.45) / 0.225
    x = self.midas(x)
    x = x - torch.min(x)
    x = x / torch.max(x)
    x = x * 255
    x = x.unsqueeze(0)
    return x


torch_model = Model(model_type)
torch_model.eval()

Using cache found in /root/.cache/torch/hub/intel-isl_MiDaS_master


Loading weights:  None


Using cache found in /root/.cache/torch/hub/rwightman_gen-efficientnet-pytorch_master


Model(
  (midas): MidasNet_small(
    (pretrained): Module(
      (layer1): Sequential(
        (0): Conv2dSameExport(3, 32, kernel_size=(3, 3), stride=(2, 2), bias=False)
        (1): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU6(inplace=True)
        (3): Sequential(
          (0): DepthwiseSeparableConv(
            (conv_dw): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
            (bn1): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
            (act1): ReLU6(inplace=True)
            (se): Identity()
            (conv_pw): Conv2d(32, 24, kernel_size=(1, 1), stride=(1, 1), bias=False)
            (bn2): BatchNorm2d(24, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
            (act2): Identity()
          )
        )
        (4): Sequential(
          (0): InvertedResidual(
            (conv_pw): Conv2d(24, 144, kernel_size=(1, 1), stri

In [ ]:
# 에러 발생하면 한번 더 실행해볼것!

#example_input = torch.rand(1, 3, 384, 384)
example_input = torch.rand(1, 3, 256, 256)
traced_model = torch.jit.trace(torch_model, example_input)
model = ct.convert(
    traced_model,
    inputs = [ct.ImageType(name = 'input_image', shape = example_input.shape)],
    outputs = [ct.Tensor(name = 'depthmap', color_layout=ct.colorlayout.GRAYSCALE)],
)
print(model.input_description)
print(model.output_description)

Translating MIL ==> NeuralNetwork Ops: 100%|██████████| 928/928 [00:01<00:00, 541.52 ops/s]


Features(input_image)
Features(depthmap)


In [ ]:
model.input_description['input_image'] = 'Input image'
model.output_description['var_1362'] = 'Inferred depth map array'

spec = model.get_spec()
ct.utils.rename_feature(spec, 'var_1362', 'depthmap')
model = ct.models.MLModel(spec)
model.save('drive/MyDrive/MiDaS-4.mlmodel')